# MTH 4320 / 5320 - Homework 1

## Gradient Descent, Linear Models, and Logistic Classification

**Deadline**: Sept 17

**Points**: 100

### Instructions

Submit **one** Python notebook file for grading. Your file must include **text explanations** of your work, **well-commented code**, and the **outputs** from your code.

### Problems

---

#### Gradient Descent

In [269]:
# imports
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd

***

1. [10 points] Write a version of gradient descent with an option to use the explicit gradient formula of your loss function OR the `estimate_gradient` approximator.

We start by defining the `estimate_gradient` function.

In [240]:
# estimate the gradient
def estimate_Gradient(f, x, h):
    n = len(x)
    gradient = np.zeros(n)
    
    for counter in range(n):
        xUp = x.copy()
        xUp[counter] += h
        gradient[counter] = (f(xUp) - f(x))/h
            
    return gradient

We then need to define a seperate function for the explicit gradient formula can mathematically be expressed as $\nabla L (\theta) = X^T X \theta - X^T y$. Note that in our code, $\theta$ will be represented by $w$. 

In [241]:
# explicitly calculate the gradient
def explicit_Gradient(x, w, y):
    # convert to np array
    X = np.array(x)
    W = np.array(w)
    Y = np.array(y)
    
    # check for 1d arrays, which can't handle transpose
    if X.ndim == 1:
        gradient = (X * X * W) - (X * Y)
    else:
        gradient = (X.T @ X @ W) - (X.T @ Y)

    return np.array(gradient)

Next, we can implement the original `gradientDescent` function, but include an additional boolean paramter `explicit` which will determine whether or not we want to use the explicit gradient formula, or default to the gradient estimate. We also need to be sure to pass the $y$ and $\theta$ values through the function so that `explicit_Gradient` can access them.  

In [242]:
# run gradient descent and output the coordinates of the estimated critical point, checking for explicit gradient
def gradientDescent(f, x0, alpha, h, tolerance, maxIterations, w, y, explicit=False):
    # set x equal to the initial guess
    x = x0 

    # take up to maxIterations number of steps
    for counter in range(maxIterations):
        # update the gradient, checking the explicit parameter to determine the gradient method
        if explicit:
            gradient = explicit_Gradient(x, w, y)
        else:
            gradient = estimate_Gradient(f, x, h)
        
        # stop if the norm of the gradient is near 0 (success)
        if np.linalg.norm(gradient) < tolerance:
            print('Gradient descent took', counter, 'iterations to converge')
            print('The norm of the gradient is', np.linalg.norm(gradient))
            
            # return the approximate critical point x
            return x
        
        # print a message if we do not converge (failure)
        elif counter == maxIterations-1:
            print("Gradient descent failed")
            print('The gradient is', gradient)
            
            # return x, sometimes it is still pretty good
            return x
        
        # take a step in the opposite direction as the gradient
        x -= alpha*gradient

Let's try this out using $x^2$ as an example. 

In [243]:
# test x^2
f = lambda x : x[0]**2

In [244]:
# using gradient estimate
x = gradientDescent(f,[2],0.4,0.4,0.000001,10000, [0.1, 0.2], [2])
print(x, f(x))

Gradient descent took 10 iterations to converge
The norm of the gradient is 4.5055999998641627e-07
[-0.19999977] 0.03999990988805076


In [245]:
# using explicit gradient
x = gradientDescent(f,[2],0.4,0.4,0.000001,10000, [0.1, 0.2], [2], explicit=True)
print(x, f(x))

Gradient descent took 16 iterations to converge
The norm of the gradient is 3.0908304814264935e-07
[19.99999985  9.99999999] 399.9999938305237


---

2. [10 points] Write a version of gradient descent that chooses $n$ random starting points and outputs the parameters resulting in minimum training loss across all the runs.

Let's add an input parameter to the`gradientDescent` function so that it selects $n$ starting points instead of just one x0 value. Then, inside the function, let's use that value $n$ to iterate through $n$ different randomly selected starting points. Finally, as we iterate through these points, lets save the best loss functions. Note: we can also remove all the previous return functions, since we do not want to output each individual run point. 

In [246]:
# run gradient descent for n random starting points, and output the parameters resulting in min training loss
def randomGradientDescent(f, x0, alpha, h, tolerance, maxIterations, w, y, n, explicit=False):
    # declare best parameters
    best_loss = float('inf')
    best_x = 0
    
    # iterate through n points
    for i in range(n):
        # randomly selecting starting value 
        x = np.random.rand(len(x0))

        # take up to maxIterations number of steps
        for counter in range(maxIterations):

            # update the gradient, checking the explicit parameter to determine the gradient method
            if explicit:
                gradient = explicit_Gradient(f, x, w, y)
            else:
                gradient = estimate_Gradient(f, x, h)

            # stop if the norm of the gradient is near 0 (success)
            if np.linalg.norm(gradient) < tolerance:
                break

            # take a step in the opposite direction as the gradient
            x -= alpha*gradient
        
        # store the current loss
        loss = f(x)
        
        # update the best parameters
        if loss < best_loss:
            best_loss = loss
            best_x = x
    
    # return the overall best values
    return best_loss, best_x   

Let's try this out using $x^2$ as an example. 

In [247]:
# using gradient estimate
loss, x = randomGradientDescent(f,[2],0.4,0.4,0.000001,10000, [0.1, 0.2], [2], 10)
print(f"The best loss was {loss}, achieved with an x value of {x}.")

The best loss was 0.03999980313185498, achieved with an x value of [-0.19999951].


---

3. [10 points] Write a version with a **cyclic** learning rate that changes in each training epoch and saves the model parameters every time the learning rate vanishes. Then, select the best parameters observed.

In [248]:
# run gradient descent for n random starting points with cyclic learning, and output the best parameters
def cyclicGradientDescent(f, x0, alpha, h, tolerance, maxIterations, w, y, n, explicit=False):
    # declare best parameters
    best_loss = float('inf')
    best_x = 0
    best_alpha = 0
    
    # iterate through n points
    for i in range(n):
        # randomly selecting starting value 
        x = np.random.rand(len(x0))
        
        # reset learning rate
        learn_rate = alpha 

        # take up to maxIterations number of steps
        for counter in range(maxIterations):

            # update the gradient, checking the explicit parameter to determine the gradient method
            if explicit:
                gradient = explicit_Gradient(f, x, w, y)
            else:
                gradient = estimate_Gradient(f, x, h)

            # stop if the norm of the gradient is near 0 (success)
            if np.linalg.norm(gradient) < tolerance:
                break

            # take a step in the opposite direction as the gradient
            x -= alpha*gradient
        
        # store the current loss
        loss = f(x)
        
        # update the best parameters
        if loss < best_loss:
            best_loss = loss
            best_x = x
            best_alpha = learn_rate
            
        # linearly decrease the learning rate
        learn_rate -= alpha / maxIterations
        
        # stop if the learning rate vanishes
        if learn_rate < 0:
            break
    
    # return the overall best values
    return best_loss, best_x, best_alpha

Let's try this out using $x^2$ as an example.

In [249]:
# using gradient estimate
loss, x, alpha = cyclicGradientDescent(f,[2],0.4,0.4,0.000001,10000, [0.1, 0.2], [2], 10)
print(f"The best loss was {loss}, achieved with an x value of {x} and a learning rate of {alpha}.")

The best loss was 0.03999981312859259, achieved with an x value of [-0.19999953] and a learning rate of 0.4.


---

4. [10 points] For the gradient descent variants from problems 2-3, instead of saving only the best parameters, save all sets of parameters that are converged to, predict outputs using each set of parameters, and then have final output predictions equal to the mean of the model predictions.

For the gradient descent in problem 2: 
\
\
Instead of storing the values for losses, let's store the value from converges when the gradient is near 0. Then, to predict the output we simply take our saved values and pass them through our `f(x)` function, and finally we return the mean.

In [250]:
# run gradient descent for n random starting points, and output the parameters resulting in min training loss
def randomMeanGradientDescent(f, x0, alpha, h, tolerance, maxIterations, w, y, n, explicit=False):
    # declare converged parameters
    converged = []
    
    # iterate through n points
    for i in range(n):
        # randomly selecting starting value 
        x = np.random.rand(len(x0))

        # take up to maxIterations number of steps
        for counter in range(maxIterations):

            # update the gradient, checking the explicit parameter to determine the gradient method
            if explicit:
                gradient = explicit_Gradient(f, x, w, y)
            else:
                gradient = estimate_Gradient(f, x, h)

            # stop if the norm of the gradient is near 0 (success)
            if np.linalg.norm(gradient) < tolerance:
                converged.append(x)
                break

            # take a step in the opposite direction as the gradient
            x -= alpha*gradient
    
    # declare predicted outputs
    predicted = [f(x)]
       
    # return the mean     
    return np.mean(predicted)

For the gradient descent in problem 3:
\
\
Let's follow the same approach as previously, where we store the converged values rather than the best loss values. Once again, finding the predicted values simply requires passing the `f(x)` function, and the mean is returned at the end. 

In [251]:
# run gradient descent for n random starting points with cyclic learning, and output the best parameters
def cyclicMeanGradientDescent(f, x0, alpha, h, tolerance, maxIterations, w, y, n, explicit=False):
    # declare converged parameters
    converged = []
    
    # iterate through n points
    for i in range(n):
        # randomly selecting starting value 
        x = np.random.rand(len(x0))
        
        # reset learning rate
        learn_rate = alpha 

        # take up to maxIterations number of steps
        for counter in range(maxIterations):

            # update the gradient, checking the explicit parameter to determine the gradient method
            if explicit:
                gradient = explicit_Gradient(f, x, w, y)
            else:
                gradient = estimate_Gradient(f, x, h)

            # stop if the norm of the gradient is near 0 (success)
            if np.linalg.norm(gradient) < tolerance:
                converged.append(x)
                break

            # take a step in the opposite direction as the gradient
            x -= alpha*gradient
            
        # linearly decrease the learning rate
        learn_rate -= alpha / maxIterations
        
        # stop if the learning rate vanishes
        if learn_rate < 0:
            break
    
    # declare predicted outputs
    predicted = [f(x)]
       
    # return the mean     
    return np.mean(predicted)

Let's try both of these out using $x^2$ as an example.

In [252]:
# using gradient estimate
mean = randomMeanGradientDescent(f,[2],0.4,0.4,0.000001,10000, [0.1, 0.2], [2], 10)
print(f"The mean of the model predictions using randomized gradient descent is {mean}.")

The mean of the model predictions using randomized gradient descent is 0.039999878350889584.


In [253]:
# using gradient estimate
mean = cyclicMeanGradientDescent(f,[2],0.4,0.4,0.000001,10000, [0.1, 0.2], [2], 10)
print(f"The mean of the model predictions using a cyclic learning rate is {mean}.")

The mean of the model predictions using a cyclic learning rate is 0.039999903404734714.


---

5. [15 points] Apply your model with all six variants of gradient descent (3 variants with estimated gradient and 3 variants with exact gradient) to predict house prices in the [Mount Pleasant Real Estate Dataset](https://www.hawkeslearning.com/Statistics/dis/datasets.html) using columns C-O, Q-T, and V-W (note some columns will need to be converted to binary or one-hot representations). Compare the results in terms of test accuracy, `fit` runtime, and `predict` runtime. 

Let's begin by importing our data from a csv file. Credit goes to Gabby Pang for providing an outline of this code on Discord for other students to use. 

In [273]:
# Read in the data as csv into a pandas file
df = pd.read_csv('/Users/andreweden/Desktop/deep_learning/Mount_Pleasant_Real_Estate_Data.csv')

# Remove NA rows
df = df.dropna(axis='rows')

# Remove unused collumns
df = df.drop(['ID','Misc Exterior', 'Amenities', 'Number of Fireplaces'], axis=1)

# Apply one-hot encoding
df = pd.get_dummies(df, columns = ['Subdivision', 'House Style'])

# Convert yes/no to binary
yes_no = ['Duplex?','New Owned?', 'Has Pool?', 'Has Dock?', 'Fenced Yard', 'Screened Porch?', 'Golf Course?', 'Fireplace?']
for i in yes_no:
    df[i].replace({"Yes": 1, "No": 0}, inplace=True)
    
# Remove commas and dollar signs
df["List Price"] = df["List Price"].str.replace(',', '').str.replace('$', '').astype(int)

In [274]:
# Print the first rows for visual confirmation
df.head()

,List Price,Duplex?,Bedrooms,Baths - Total,Baths - Full,Baths - Half,Stories,Square Footage,Year Built,Acreage,...,House Style_Colonial,House Style_Condo Regime,House Style_Condominium,House Style_Contemporary,House Style_Cottage,House Style_Craftsman,House Style_Patio,House Style_Ranch,House Style_Townhouse,House Style_Traditional
0,369900,1,3.0,2.5,2.0,1.0,2.0,1797.0,2017.0,0.06,...,False,False,False,False,False,False,False,False,True,False
1,375000,1,3.0,2.5,2.0,1.0,2.0,1797.0,2017.0,0.06,...,False,False,False,False,False,False,False,False,True,False
2,769900,0,4.0,3.5,3.0,1.0,2.0,2767.0,2014.0,0.35,...,False,False,False,False,False,False,False,False,False,True
3,699990,0,4.0,3.5,3.0,1.0,2.0,3240.0,2014.0,0.29,...,False,False,False,False,False,False,False,False,False,True
4,436625,0,4.0,3.0,3.0,0.0,2.0,2072.0,2017.0,0.19,...,False,False,False,False,False,False,False,False,False,True


Now we must define our 'fit' and 'predict' functions, which we can store in a logistic classifier class. 

In [271]:
class LogisticClassifierGradient:
    
    # initialize and select the gradient type
    def __init__(self, gradientType=None, explicit=False):
        self.gradientType = gradientType
        self.explicit = explicit
        
    # fit the model to the data
    def fit(self, X, y, w0, alpha, h, tolerance, max_iterations):

        # save the training data
        X = np.hstack((np.ones([X.shape[0], 1]), X))

        # standardize the data
        X = scale(X)

        # find the w values that minimize the sum of squared errors via gradient descent
        L = lambda w: (expit(X @ w).T - y.T) @ (expit(X @ w) - y)
        
        # find gradient based on gradient type
        if self.gradientType == 'random':
            randomGradientDescent(L, w0, alpha, h, tolerance, maxIterations, w0, y, n, explicit=self.explicit)
        elif self.gradientType == 'cyclic':
            cyclicGradientDescent(L, w0, alpha, h, tolerance, maxIterations, w0, y, n, explicit=self.explicit)
        else:
            gradientDescent(L, w0, alpha, h, tolerance, maxIterations, w0, y, explicit=self.explicit)
            

    # predict the output from testing data
    def predict(self, X):        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))

        # standardize the data
        X = scale(X)

        # return the predicted y's
        return expit(X @ self.w)

Finally, let's put this all together with our past gradient descent methods. 

In [276]:
# find the data and labels
X = df['List Price']
Y = df.drop('List Price', axis=1)

# do a train/test split 
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.25)

# build the logistic classifier
model = LogisticClassifierGradient()

# fit the Bayes classifier to the training data 
model.fit(trainX, trainY,
          [0] * (X.shape[1] + 1), alpha=0.001, h=0.001,
          tolerance=0.001, max_iterations=10000)

# predict the Labels of the training set 
predictedY = np.rint(model.predict(trainX))

# print quality metrics 
print(f'\nTrain Classification Report: \n {classification_report(trainY, predictedY)}')

# predict the Labels of the test set 
predictedY = np.rint(model.predict(testX))

# print quality metrics 
print(f'Test Classification Report: \n {classification_report(testY, predictedY)}')

print('Train Confusion Matric: \n')

sn.heatmap(confusion_matrix(testY, predictedY), annot=True)

IndexError: tuple index out of range

I don't know what I'm doing wrong at this point. However, if I had this working, I would call the `LogisticClassifierGradient` 6 times in total, each time initializing with respect to a different gradient method, and selecting between explicit or estimated gradient. 

---

#### Logistic Classification

In [254]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn

from scipy.stats import multivariate_normal
from scipy.special import expit
from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

---

6. [10 points] Write a multi-class version of the `BinaryLogisticClassifier` from lecture in the style of `scikit-learn` classifiers (i.e. as a Python class with `fit` and `predict` functions), optimized by gradient descent.

Let's begin by redefining our `BinaryLogisticClassifier`, for which I will be using the equivalent `LogisticClassifierGradient` from the class github notebook `Week-3.ipynb`.

In [255]:
class BinaryLogisticClassifier:
        
    # fit the model to the data
    def fit(self, X, y, w0, alpha, h, tolerance, max_iterations):
        
        # save the training data
        X = np.hstack((np.ones([X.shape[0], 1]), X))
        
        # standardize the data
        X = scale(X)
        
        # find the w values that minimize the sum of squared errors via gradient descent
        L = lambda w: (expit(X @ w).T - y.T) @ (expit(X @ w) - y)
        self.w = self.gradientDescent(L, w0, alpha, h, tolerance, max_iterations)
                
    # predict the output from testing data
    def predict(self, X):        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # standardize the data
        X = scale(X)
        
        # return the predicted y's
        return expit(X @ self.w)
    
    # run gradient descent to minimize the loss function
    def gradientDescent(self, f, x0, alpha, h, tolerance, max_iterations):
        # set x equal to the initial guess
        x = x0

        # take up to maxIterations number of steps
        for counter in range(max_iterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)

            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                
                # return the approximate critical value x
                return x

            # if we do not converge, print a message
            elif counter == max_iterations - 1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient
            x -= alpha*gradient
            
    # estimate the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)
        
        # compute f at current point
        fx = f(x)

        # find each component of the gradient
        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - fx)/h

        # return the gradient
        return gradient

Next, we need to define a new multi-class verson, that will call upon this previous `BinaryLogisticClassifier` over multiple iterations. Within this class we need to create the following:
1. A fit function that includes multiple classes instead of just one, and iterates over each class
2. A predict function. that iterates over each classifier and selects the highest probability

In [256]:
class MultiClassLogisticClassifier:
   # fit the model to the data
    def fit(self, X, y, w0, alpha, h, tolerance, max_iterations):
        # declare list of binary classifiers
        self.classifiers = []
        
        # find the number of unique classes
        num_classes = len(np.unique(y))
        
        # iterate over each class for traning and fitting
        for i in range (num_classes):
            # create a binary label vector for the current class
            label = LabelBinarizer(neg_label=0, pos_label=1)
            y_binary = label.fit_transform(y)
            y_binary = y_binary[:, i]
            
            # train and store binary classifier 
            classifier = BinaryLogisticClassifier()
            classifier.fit(X, y_binary, w0, alpha, h, tolerance, max_iterations)
            self.classifiers.append(classifier)
    
    # predict the output
    def predict(self, X):
        # declare array for class probabilities
        probabilities = np.zeros((X.shape[0], len(self.classifiers)))
        
        # iterate over each classifier and update probabilities
        for i in range(len(self.classifiers)):
            probabilities[:,i] = self.classifiers[i].predict(X)
            
        # select the class with the highest probability 
        prediction = np.argmax(probabilities, axis=1)
        
        return prediction  

---

7. [10 points] Derive a formula for the gradient of the MSE loss function with respect to the weights of your multi-class logistic classifier.

The MSE loss function can be expressed as follows:
$MSE = \frac{1}{n}\sum_{n=1}^{n} (\hat{Y}_i - Y_i)^2$

The gradient can be found by taking the derivative, which yields: $MSE = \frac{1}{n}\sum_{n=1}^{n} (\hat{Y}_i - Y_i)$

However, this only applies to a single logistic classifier, so for a multi-class classifier it is necessary to average over all the weights.

I don't know how to do this. 

---

8. [10 points] Add functionality to optimize models using the three variants of gradient descent from Problems 2-4.

Add the other gradient descent variants as options, much like problem 5. 

---

9. [15 points] Apply your classifier and all six variants of gradient descent (3 variants with estimated gradient and 3 variants with exact gradient) to classify the MNIST dataset. Compare the results in terms of test accuracy, `fit` runtime, and `predict` runtime.

In [270]:
# Importing MNIST
mnist = datasets.load_digits()
X = mnist.data.astype(float)
y = mnist.target.astype(int)

Once the data is imported, run the classifiers much like problem 5. 